# Create the Dataset
Use the whisper API to get transcipts and then use ChatGPT to get if its interesting or not.

In [ ]:
%pip install pytube
%pip install youtube-dl
%pip install openai
%pip install moviepy

In [5]:
import config

import pytube

import requests
import subprocess
import os
import shutil
import ast

import openai

from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.audio.io.AudioFileClip import AudioFileClip

import youtube_dl

In [6]:
openai.api_key = config.OAI_API_TOKEN

In [ ]:
# YouTube video URL
vid_url = "https://www.youtube.com/watch?v=1aA1WGON49E"
fps = 30

In [ ]:
# Get the audio stream
audio_stream = pytube.YouTube(vid_url).streams.filter(only_audio=True).first()
# Check if audio stream is not None before attempting to download it
if audio_stream is not None:
    # Download the audio stream
    audio_file = audio_stream.download(filename="input.mp4").replace("\\","/")
else:
    print("No audio stream found.")


Break the video down into 25mb chunks

In [ ]:
def split_file(input_file_path, chunk_size):
    """
    Splits a file into smaller chunks and saves them to disk.
    """
    with open(input_file_path, 'rb') as input_file:
        for chunk_num, chunk in enumerate(iter(lambda: input_file.read(chunk_size), b'')):
            # Save the chunk to disk
            output_file_path = f"videos/input_{chunk_num}.mp4"
            with open(output_file_path, 'wb') as output_file:
                output_file.write(chunk)

newpath = os.getcwd() + "/videos/"
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
# Usage example
input_file_path = 'input.mp4'
chunk_size = 25 * 1024 * 1024  # 25MB in bytes
split_file(input_file_path, chunk_size)

Create metadata for the files <br/>
<sub>Start time and duration</sub>

In [ ]:
newpath = os.getcwd() + "/metadata/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

for i, filename in enumerate(os.listdir('videos')):
    start_time = 0.0
    md_output_file_path = f"metadata/input_{i}.txt"
    with open(md_output_file_path, 'w') as output_file:
        clip = AudioFileClip('videos/'+filename)
        duration = clip.duration
        clip.close()
        output_file.write(f"[{start_time},{duration}]")
        start_time += duration

Input the audio into Whisper and get transcription


In [ ]:
newpath = os.getcwd() + "/chunks/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

url = 'https://api.openai.com/v1/audio/transcriptions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}
data = {
    'model':'whisper-1',
    'prompt':'Woah! I for a fact don\'t believe that... Okay, I know. I love those~'
}
for filename in os.listdir('videos'):
    chunk_num = 0
    with open("videos/" + filename, "rb") as input_file:
        files = {
            'file':(filename,input_file)
        }
        transcription = requests.post(url=url, headers=headers, files=files, data=data)
        output_file_path = f"chunks/input_{chunk_num}.txt"
        with open(output_file_path, 'w') as output_file:
            output_file.write(transcription.json().get('text'))
        chunk_num += 1

Break down the texts into workable chunks

In [ ]:
# TO DO

```
Break this pragraph down into chunks (not specifically sentences but chunks of text that seem to be in the same coherence) that seem reasonable and then split those into quotes and seperate those by commas and form a python list. Only spit out the list format for the chunks (make sure there are no newlines or whitespaces).
```

Get sentiment from ChatGPT and delete files that are negative

In [ ]:
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}
for filename in os.listdir('chunks'):
    with open("chunks/" + filename, "r") as input_file:
        data = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": f"respond with either 'pos' or 'neg' to if this sentence is entertaining or not in any definition you find as an AI model.\n\"{input_file.read()}\""}] 
        }
        response = requests.post(url=url, headers=headers, json=data)
        sentiment = response.json().get('choices')[0].get('message').get('content')
        if sentiment == 'neg':
            if os.path.exists("metadata/" + filename):
                os.remove("metadata/" + file_path)

Create Video from choosen segments

In [10]:
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}
data = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": f"using python and ffmpeg through sub processes get a start and a duration and get clips from a video and then combine them"}] 
        }
response = requests.post(url=url, headers=headers, json=data)
sentiment = response.json().get('choices')[0].get('message').get('content')

In [11]:
print(sentiment)



As an AI language model, I do not have access to the ffmpeg, so I will only provide you with a general outline of the procedure.

Firstly, you can use the Python subprocess module to execute the ffmpeg commands. You’ll need to install the ffmpeg command-line tool on your machine for this.

Secondly, you can use the ‘-ss’ and ‘-t’ commands to define the start and duration of each clip, respectively. For example, to get a 10-second clip starting at 30 seconds into a video named “video.mp4”, you’d use the following command:

```
ffmpeg -ss 00:00:30 -i video.mp4 -t 00:00:10 -c copy clip1.mp4
```

This would create a new video file called “clip1.mp4” that contains the 10-second clip.

You can then repeat this for each clip you want to create.

Finally, you can use the ‘concatenate’ command to join the individual clips into one complete video. For example, to join two clips named “clip1.mp4” and “clip2.mp4” into a single video named “output.mp4”, you’d use the following command:

```
ffmpe

Delete directories

In [ ]:
newpath = ["/metadata/","/videos/","/chunks/"]
for i in newpath:
    if os.path.exists(os.getcwd() + i):
        shutil.rmtree(os.getcwd() + i)